# Documentation (EN)

**Contents**
- 1. Introduction & Overview
- 2. UI & Level Design
- 3. Game Logic
- 4. Summary & Outlook


# 1. Introduction & Overview
- Game idea: JavaFX 2D platformer. Collect USB stick, secure folder files, avoid hazards, reach portal.
- Architecture: `Game` (bootstrap, config, sound, level), `ScreenManager` (window, input, game loop + delta), `GameScreen` (rendering, physics, camera, HUD, state transitions), `Level` + `TutorialLevel`/`SecondLevel`, `Platform` + `Block` variants, `EntityPlayer`.
- Pipeline:
```text
Input -> InputManager -> GameScreen.update(delta)
  -> Physics (dx/dy, gravity, collision)
  -> Camera (dead zone + smoothing)
  -> Level objects relative to camera
  -> HUD (hearts, files, debug)
```


# 2. UI & Level Design
## 2.1 Scene & Rendering
- `initialize()`: clear root, animated background, place player, load/draw platforms/blocks, HUD + buttons + pause overlay.
- `update(delta)`: physics, input, collisions, camera dead zone, HUD refresh.
- `Platform.drawPlatform()`: tiled floor with camera offset.
- `Block.update()`: move sprites by camera.

## 2.2 HUD & Score
- Files/score: `countFolderBlocks()` vs `countActiveFolders()` -> `Files: X/Y` label.
- Quest/tooltips: `questLbl`, `tipsLbl` (F1), `flipperHint` after getting flipper.
- Health: `drawHealth()` hearts; debug bar (F2); back/pause buttons.

## 2.3 Moving / Changing Objects
- Player: dx/dy, gravity, jump power, direction sprite, F3 NoClip/GodMode.
- Platforms: `FloatingPlatformBlock` ping-pongs with carry-over; `EntityMovingPlatform` hover/sinus.
- Enemies/projectile: `RobotEnemyBlock` patrol + `LaserBlock`; `BossRobotBlock` fires only if player above. Laser lifetime 4s, 1 damage (ignored in GodMode).
- Hazards/perks: `LavaBlock` tick damage; `GasBarrierBlock` disabled via E + Flipper (fade); `JumpBoost`/`SpeedBoost` 10s buff; `USBStick` unlocks collection; `FolderBlock` counts after unlock.

## 2.4 Game States
- Start: `Game.start()` -> loading -> menu -> GameScreen.
- Pause: `togglePause()` overlay (logic still running; TODO real halt).
- Game over: HP <= 0 or fall -> MainMenu.
- Finish: `FinishBlock.onCollide` -> `LevelFinishedScreen` -> ?Next Level? sets `SecondLevel` and new GameScreen.

## 2.5 Level Design
- Level 1: ground + lava gaps, USB stick, jump/speed boosts, floating platform, finish; tutorial flag in `onFinish()`.
- Level 2: denser platforms, lava columns, 8 folder blocks, jump-boost route, flipper + gas barrier, boss robot, finish; one-time spawn sound.
- Condition: finish collision; USB stick required for files.


# 3. Game Logic
## 3.1 Goal & Flow
- Goal: unlock collection (USB), gather files, reach finish alive.
- Start: menu -> GameScreen, HP 5 hearts.
- End: HP 0, fall, or finish collision.

## 3.2 Input & Interaction
- Keys: A/D move, SPACE jump (grounded), E interact (barrier), W/S in NoClip, F1/F2/F3 toggles.
- InputManager: `isPressed` for held keys, `pollJustPressed` for debounced toggles.

## 3.3 Events & Logic
- Collision: AABB, order platforms then blocks; `pendingBlocks` for lasers.
- Damage: clamped HP, GodMode ignores; lava/enemy/laser apply damage, stomp kills enemy.
- Timing: delta-based movement; boosts reset via `PauseTransition(10s)`; lasers 4s; `levelStarted` for stats.
- Separation: logic in GameScreen/blocks, rendering via UI utils; camera offsets in `update()`.

## 3.4 Classes (snapshot)
- `Game`, `ScreenManager`, `GuiScreen`, `GameScreen`.
- `Level`, `TutorialLevel`, `SecondLevel`, `Platform`.
- `EntityPlayer`, `FloatingPlatformBlock`, `EntityMovingPlatform`.
- `RobotEnemyBlock`, `BossRobotBlock`, `LaserBlock`.
- `LavaBlock`, `GasBarrierBlock`, `JumpBoostBlock`, `SpeedBoostBlock`, `USBStickBlock`, `FolderBlock`, `FlipperItem/Block`, `FinishBlock`.


# 4. Summary & Outlook
- Stable core: GameScreen + ScreenManager + Level with delta physics, dead-zone camera, HUD/debug.
- Requirements covered: score (files), visible states (HUD/overlays), two levels, moving objects.
- Next: real paused logic, use Level.update for movers, richer HUD (timer/score), checkpoints, persistent stats, per-level camera clamps.
